In [9]:
import pandas as pd
answers = open('./Answers.txt', 'w')

tweets =  pd.read_csv('./archive/2010.csv', index_col=False)
tweets
x = tweets.shape[0]
for i in range(2011, 2022):
    more_tweets = pd.read_csv(f'./archive/{str(i)}.csv', index_col=False)
    x += more_tweets.shape[0]
    tweets = pd.concat([tweets, more_tweets], ignore_index=True)

tweets.drop(['Unnamed: 0'], axis=1, inplace=True)
tweets.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,15434727182,15434727182,1.275676e+12,2010-06-04 18:31:57,0,NaN,"Please ignore prior tweets, as that was someon...",en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,152153637639028736,152151847614943233,1.325111e+12,2011-12-28 22:27:08,0,NaN,@TheOnion So true :),en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,151809315026636800,151809315026636800,1.325029e+12,2011-12-27 23:38:55,0,NaN,If you ever wanted to know the *real* truth ab...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,151338939389706242,151338939389706242,1.324917e+12,2011-12-26 16:29:50,0,NaN,Walked around a neighborhood recently rebuilt ...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,151337237429239808,151337237429239808,1.324917e+12,2011-12-26 16:23:04,0,NaN,"It was Xmas, so we brought presents for the ki...",en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [10]:
# Question 1
num_tweets = tweets.shape[0]
avg_nlikes = tweets['nlikes'].sum() / num_tweets
avg_nlikes = int(avg_nlikes)
answers.write(str(avg_nlikes) + '\n')

avg_nlikes

14705

In [11]:
# Question 2
total_views = (tweets['nlikes'].sum() + tweets['nreplies'].sum() + tweets['nretweets'].sum()) / 0.05
avg_view = total_views / num_tweets
avg_view = int(avg_view)
answers.write(str(avg_view) + '\n')

avg_view

338864

In [ ]:
# Question 3
from requests import get
from io import BytesIO
from PIL import Image

# get total number of char's in the tweet
tweet = tweets['tweet']
total_len_tweets = 0
for t in tweet:
    total_len_tweets += len(t)

# get total image size
def parse_url(photo_urls):
    urls = photo_urls[1:-1].split(', ')
    urls = list(map(lambda x:x[1:-1], urls))
    return urls

photos = tweets['photos']
photo_size_bytes = 0
NUM_SAMPLE_TWEETS = 1500
total_num_photo = 0
for i, photo in enumerate(photos):
    if photo == '[]': 
        continue
        
    photo_urls = parse_url(photo)
    total_num_photo += len(photo_urls)

    #look at first 500, mid 500, last 500 tweets to estimate average photo size per tweet
    if i < 100 or 20100 < i <= 2000 or  i >= len(photos) - 1000:
        print(i)
        for url in photo_urls:
            response = get(url)
            try:
                img = Image.open(BytesIO(response.content))
                width, height = img.size
                # jpg images has 3 bytes / pixel and photos are all jpg
                photo_size_bytes += width * height * 3 
            except:
                pass

avg_photo_size = photo_size_bytes / NUM_SAMPLE_TWEETS
total_photo_size_estimate = avg_photo_size * total_num_photo

# get total video size
VIDEO_SIZE = 256 * 2^20 # max Twitter video size is 512MB, so we assume average video size is 256MB
num_total_videos = 0
for video in tweets['video']:
    num_total_videos += video
total_video_size_estimate = VIDEO_SIZE * num_total_videos

total_data_size = total_len_tweets + total_photo_size_estimate + total_video_size_estimate
avg_data_size_per_tweet = total_data_size / len(tweets)
avg_data_size_per_tweet = int(avg_data_size_per_tweet)
answers.write(str(avg_data_size_per_tweet) + '\n')

avg_data_size_per_tweet

3
4
11
12
13
26
30
33
56
67
78
83
85
88
93
96
101
103
111
127
128
142
146
158
159
164
168
170
202
208
211
217
228
234
235
238
240
242
246
248
252
253
258
261
262
263
268
271
284
291
316
317
324
325
326
339
346
352
374
392
397
403
404
412
420
424
426
436
465
481
484
20501
20567
20570
20588
20596
20599
20611
20651
20656
20695
20715
20722
20728
20752
20799
20828
20863
20866
20867
20868
20869
20870
20908
20951
20952
20966
20974
20982
42086
42093
42094
42103
42122
42123
42124
42125
42131
42147
42169
42170
42174
42190
42199
42200
42202
42204
42205
42206
42207
42212
42218
42220
42248
42280
42302
42303
42304
42320
42322
42329
42336
42342
42348
42370
42388
42393


In [ ]:
#Question 4
total_data_transfer = avg_data_size_per_tweet * total_views
total_data_transfer = int(total_data_transfer)
answers.write(str(total_data_transfer) + '\n')

total_data_transfer

In [ ]:
#Question 5
TWELVE_YEARS_IN_MIN = 11 * 365 * 24 * 60
avg_views_per_min = TWELVE_YEARS_IN_MIN / total_views
avg_views_per_min = int(avg_views_per_min)

avg_views_per_min

In [ ]:
data_transfer_per_min = avg_views_per_min * avg_data_size_per_tweet

In [ ]:
answers.close()